In [ ]:
# 딥 러닝을 이용한 자연어 처리 입문

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 64.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 26.8 MB/s eta 0:00:00


In [ ]:
import os, copy
import pandas as pd
import numpy as np
from tqdm import tqdm
import urllib.request
from sklearn import preprocessing
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel, TFBertForSequenceClassification
from keras.callbacks import EarlyStopping, ModelCheckpoint
import warnings
from google.colab import drive

warnings.filterwarnings('ignore')
drive.mount("/content/Drive/", force_remount=True)

Mounted at /content/Drive/


In [ ]:
origin_df = pd.read_csv('/content/Drive/MyDrive/NP_classifier/mod_intent_model_dataset.csv')
df = copy.deepcopy(origin_df)
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df.drop(df.loc[df['sentence'].str.contains('\*')].index, axis=0, inplace=True)
df = df.loc[df['sentence'].str.len() <= 100]
df.reset_index(inplace=True, drop=True)

df.loc[((df['sentence'].str.contains('나는')) & df['sentence'].str.contains('가 좋아')) & (df['intent']!='(표현) 긍정감정 표현하기'), 'intent'] = '(표현) 긍정감정 표현하기'
df.loc[((df['sentence'].str.contains('나는')) & df['sentence'].str.contains('이 좋아')) & (df['intent']!='(표현) 긍정감정 표현하기'), 'intent'] = '(표현) 긍정감정 표현하기'
df.loc[(df['sentence'].str.contains('응') & df['sentence'].str.contains('좋아')) & (df['intent']!='(표현) 긍정감정 표현하기'), 'intent'] = '(표현) 긍정감정 표현하기'
df.loc[(df['sentence'].str.contains('응') & df['sentence'].str.contains('맞아')) & (df['intent']!='(표현) 긍정감정 표현하기'), 'intent'] = '(표현) 긍정감정 표현하기'

df

,sentence,intent
0,애덜앙 나 너무 배불러서 배 아파,(단언) 진술하기
1,배 아프면 약국 가서 소화제라도 마시는 거 어때,(지시) 충고/제안하기
2,떡볶이랑 튀김이랑 닭껍질튀김이랑 마카롱 먹었어,(단언) 진술하기
3,아닌데 그냥 많이 먹은 거 같은데,(단언) 주장하기
4,그냥 나 뭐 사주기 싫어서 그런 거 같은데.,(단언) 주장하기
...,...,...
1142022,매물 위치와 평수를 알 수 있을까요?,(지시) 질문하기
1142023,잘 진행되고 있나요?,(지시) 질문하기
1142024,"네, 안녕하세요?",(표현) 인사하기
1142025,네~ 안녕하세요,(표현) 인사하기


In [ ]:
asdf = list(df['intent'].unique())
for i in asdf :
    print(i,df.loc[df['intent']==i,'intent'].count())

(단언) 진술하기 347226
(지시) 충고/제안하기 34844
(단언) 주장하기 437318
(지시) 질문하기 273903
(지시) 부탁하기 5029
(단언) 반박하기 3261
(표현) 감사하기 1623
(표현) 사과하기 552
(지시) 명령/요구하기 3830
턴토크 사인(관습적 반응) 4101
(언약) 약속하기(제3자와)/(개인적 수준) 14153
(표현) 긍정감정 표현하기 8769
(표현) 부정감정 표현하기 6328
(언약) 거절하기 480
(표현) 인사하기 503
(선언/위임하기) 107


In [ ]:
tokenizer = BertTokenizer.from_pretrained("klue/bert-base")

In [ ]:
# token 바구니
N_count = []

# 필요 문장 추출
N_intent = ['(표현) 부정감정 표현하기', '(단언) 반박하기', '(언약) 거절하기']

# 필요 문장 구성하는 모든 token 바구니에 넣기
for intent in N_intent :
    for i in df.loc[(df['intent'] == intent), 'sentence']:
        N_count += tokenizer.encode(i)

# token 고유값과 개수 / N_uni[0] = 고유값 리스트 / N_uni[1] = 고유값 개수 리스트
N_uni = np.unique(N_count, return_counts=True)

# 고유값 개수 내림차순 정렬
N_cnt = sorted(list(set(N_uni[1])),reverse=True)

# N_many = N_uni[0]에서 검색할 index값 / N_token = 사용 횟수가 많은 token
N_many = []
N_token = []

for i in N_cnt:
    if i > 200 :
        N_many.append(list(N_uni[1]).index(i))
for j in range(len(N_many)) :
    N_token.append(N_uni[0][N_many[j]])
print(N_many)

tokenizer.decode(N_token)

[1, 12, 1069, 893, 949, 927, 751, 946, 892, 912, 902, 948, 926, 1863, 518, 170, 26, 996, 1043, 1083, 942, 1025, 1024, 5248, 2202, 1746, 977, 3320, 909, 935, 813, 1006, 616, 79, 1004, 1117, 516, 545, 914, 86, 1238, 1531, 3, 173, 867, 881, 2303, 91, 10, 4423, 649, 183, 1151, 2197, 5244, 520, 233, 1765, 48, 1104, 352, 0, 1003, 312, 344, 1997, 577, 5872, 2577, 1028, 1744, 2319, 1198, 2516]


'[CLS].는이도키 키가어은다지고 너무 안 나?에아야서하네는데 진짜 아니데 ㅠㅠ을해 하요 있 같거게 아 없나 거면겠! 난기들 힘들 것,잖아 좀 내었 근데 싫어 않 더 그런 ~까 뭐 [UNK]인 말 못 정말 왜 맞아ㅠㅠ니 사람 걱정던 그래도'

In [ ]:
# token 바구니
P_count = []

# 필요 문장 추출
P_intent = ['(표현) 긍정감정 표현하기','(표현) 감사하기']

# 필요 문장 구성하는 모든 token 바구니에 넣기
for intent in P_intent :
    for i in df.loc[(df['intent'] == intent), 'sentence']:
        P_count += tokenizer.encode(i)

# token 고유값과 개수 / N_uni[0] = 고유값 리스트 / N_uni[1] = 고유값 개수 리스트
P_uni = np.unique(P_count, return_counts=True)

# 고유값 개수 내림차순 정렬
P_cnt = sorted(list(set(P_uni[1])),reverse=True)

# N_many = N_uni[0]에서 검색할 index값 / N_token = 사용 횟수가 많은 token
P_many = []
P_token = []

for i in P_cnt:
    if i > 200 :
        P_many.append(list(P_uni[1]).index(i))
for j in range(len(P_many)) :
    P_token.append(P_uni[0][P_many[j]])
print(P_many)

tokenizer.decode(P_token)

[1, 11, 1025, 564, 877, 702, 845, 5697, 2899, 150, 901, 3, 844, 899, 609, 895, 900, 853, 876, 946, 864, 4492, 996, 885, 1786, 977, 976, 574, 48, 1038, 1074, 958, 2130, 860, 74, 27, 6224, 6754, 5063, 4131, 524, 579, 79, 956, 3396, 756, 1200, 2069, 484, 1193, 925, 6489, 829, 84, 5682, 1107, 1928, 9, 482, 3274, 1468, 1421, 954, 3226, 152, 199, 160, 930, 3560, 6510, 919, 3791, 2457, 1718]


'[CLS].는 응키 키이 맞아 좋아 나도!어가 좋서지다고에은 고마워아해 너무하네 있 ~야게요 진짜을 같?더라 응응는데 하하 오 잘 거거 좋아하 하한 감사 안면데 좋아해들 것겠다었 정말, 아 고맙겠줘보 ㅠㅠ 난 다 내만 재밌에서를 대박 엄청 그래'

In [ ]:
# token 바구니
M_count = []

# 필요 문장 추출
M_intent = ['(단언) 진술하기', '(지시) 충고/제안하기', '(단언) 주장하기', '(지시) 질문하기', '(지시) 부탁하기',
       '(단언) 반박하기', '(표현) 사과하기', '(지시) 명령/요구하기', '턴토크 사인(관습적 반응)',
       '(언약) 약속하기(제3자와)/(개인적 수준)', '(표현) 인사하기', '(선언/위임하기)']

# 필요 문장 구성하는 모든 token 바구니에 넣기
for intent in M_intent :
    if df.loc[(df['intent'] == intent), 'sentence'].count() > 1000:
        for i in (df.loc[(df['intent'] == intent), 'sentence'].sample(n=1000)):
            M_count += tokenizer.encode(i)
    else:
        for i in (df.loc[(df['intent'] == intent), 'sentence']):
            M_count += tokenizer.encode(i)

# token 고유값과 개수 / N_uni[0] = 고유값 리스트 / N_uni[1] = 고유값 개수 리스트
M_uni = np.unique(M_count, return_counts=True)

# 고유값 개수 내림차순 정렬
M_cnt = sorted(list(set(M_uni[1])),reverse=True)

# N_many = N_uni[0]에서 검색할 index값 / N_token = 사용 횟수가 많은 token
M_many = []
M_token = []

for i in M_cnt:
    if i > 200 :
        M_many.append(list(M_uni[1]).index(i))
for j in range(len(M_many)) :
    M_token.append(M_uni[0][M_many[j]])
print(M_many)

tokenizer.decode(M_token)

[1, 13, 933, 759, 28, 1075, 955, 1002, 900, 899, 953, 932, 954, 180, 941, 3, 919, 1030, 618, 1090, 1124, 949, 53, 908, 650, 1480, 520, 91, 522, 824, 1029, 1048, 1011, 830, 1247, 981, 5306, 1531, 625, 72, 1010, 987, 337, 653, 1734, 190, 1570, 2198, 4328, 2233, 921, 83, 986, 1253, 884, 326, 1007, 1783, 3338, 1984, 550, 1044, 607, 1111, 197, 567, 1736, 230, 973, 1159, 6836, 1175, 11, 3230, 1926, 394, 356, 6556]


'[CLS].키 키?는도에이어가고지 나해!은하 있야게서 ~다 좀줘 아 거 안 하네아요 해면데는데겠 잘 가라자 먹 좋 아니 내봐 진짜 하하 그럼나 같만한들 말보 그래 ㅠㅠ 알려 없주 응까 너 오 우리 다를었에서랑,으로 같이 보 뭐더라'

In [ ]:
# 중복값 제거
print(tokenizer.decode((set(N_token).difference(set(P_token))).difference(set(M_token))))
print(tokenizer.decode((set(P_token).difference(set(N_token))).difference(set(M_token))))

근데 [UNK]니인 걱정 않 힘들잖아ㅠㅠ 싫어던 그래도 왜 그런기 사람 못 더
고맙 고마워 좋아 감사 응응 좋아하 재밌 대박 엄청 좋아해겠다


In [ ]:
N_keyword = ['싫어\.\.', '아니', 'ㅠㅠ', 'ㅜㅜ']
P_keyword = ['감사합', '고맙', '고마워', '하하', '대박', ' 좋다', '좋아!', 'ㅋㅋ', '그래\.', '그래,', '맞아', '응응', '응\.', '응,']

In [ ]:
for intent in P_intent:
    print(len(df.loc[df['intent']==intent]))
    for key in N_keyword:
        df.drop(df.loc[(df['intent'] ==intent)&df['sentence'].str.contains(key)].index, axis=0, inplace=True)  
    print(len(df.loc[df['intent']==intent]))

8769
8364
1623
1536


In [ ]:
for intent in N_intent:
    print(len(df.loc[df['intent']==intent]))
    for key in P_keyword:
        df.drop(df.loc[(df['intent'] ==intent)&df['sentence'].str.contains(key)].index, axis=0, inplace=True)  
    print(len(df.loc[df['intent']==intent]))

6328
5960
3261
3142
480
457


In [ ]:
temp = pd.DataFrame()

df_ = copy.deepcopy(df)

for intent in (N_intent+P_intent):
    df_.drop(df_[df_['intent'] == intent].index, axis=0, inplace=True)

temp = pd.concat([temp,df_.sample(n=20000)], ignore_index=True)

print(len(temp))
for key in (N_keyword+P_keyword):
    temp.drop(temp.loc[temp['sentence'].str.contains(key)].index, axis=0, inplace=True)

print(len(temp))

for intent in (N_intent+P_intent):
    temp = pd.concat([temp,df[df['intent'] == intent]], ignore_index=True)

print(len(temp))

temp.loc[(temp['intent'] == '(표현) 부정감정 표현하기')|(temp['intent'] == '(언약) 거절하기')|
         (temp['intent'] == '(단언) 반박하기'), 'intent'] = '부정'
temp.loc[(temp['intent'] == '(표현) 긍정감정 표현하기')|(temp['intent'] == '(표현) 감사하기'), 'intent'] = '긍정'
temp.loc[(temp['intent'] != '부정')&(temp['intent'] != '긍정'), 'intent'] = '중립'

20000
17100
36559


In [ ]:
df = copy.deepcopy(temp)

In [ ]:
lable_num = len(df['intent'].unique())
lable_num

3

In [ ]:
intet_label = list(df['intent'].unique())

label_dict = {0: '긍정', 1: '부정', 2: '중립'}

for idx, intent_lab in label_dict.items() :
    df.loc[df['intent'] == intent_lab, 'intent'] = idx

print(label_dict)
df = df.sample(frac=1).reset_index(drop=True)
df

{0: '긍정', 1: '부정', 2: '중립'}


,sentence,intent
0,해야죠...,2
1,나는 요리 잘하는 남편이 되긴 틀린 거 같아,2
2,정말 너무 춥다.,0
3,오오 키키 그러면 뭐... 지각만 안 하면 괜찮겠는데요?,0
4,오 추천 땡큐 땡큐! 키키,0
...,...,...
36554,3억 짜리면 300정도 키키,2
36555,그러니까... 너무 좋아 돈 생겼다~=,0
36556,우리 보컬 학원 다녀 볼래?,2
36557,거기서도 딱 목소리가 들리고 좋아,2


In [ ]:
train_data = pd.DataFrame()

len_ = df['intent'].count()

train_data = pd.concat([train_data,df.loc[:(len_*8)/10]],ignore_index=True)
print(train_data)

                              sentence intent
0                               해야죠...      2
1             나는 요리 잘하는 남편이 되긴 틀린 거 같아      2
2                            정말 너무 춥다.      0
3      오오 키키 그러면 뭐... 지각만 안 하면 괜찮겠는데요?      0
4                       오 추천 땡큐 땡큐! 키키      0
...                                ...    ...
29243                  너네가 있어서 너무 든든해~      0
29244              우리만 조심하느라 집에만 있는거지.      2
29245                       키키 아 너무 웃겨      0
29246                 사자는 무서워서 못 키우겠어,      1
29247           진짜 영수증 없었으면 바가지 쓸 뻔 했네      1

[29248 rows x 2 columns]


In [ ]:
test_data = pd.DataFrame()

len_ = df['intent'].count()

test_data = pd.concat([test_data,df.loc[(len_*8)/10:]],ignore_index=True)
print(test_data)

                                  sentence intent
0     난 아직도 가지 안 해봤졍 혼자 사니까 귀찮아서 김밥은 절레 절레      1
1                  나도 야구는 보는 것 만 좋아 난 한화인데      0
2                           키키 그냥 모든 동물 키키      2
3                                 아니 그럴 리가      1
4                                    넘 무서워      1
...                                    ...    ...
7306                       3억 짜리면 300정도 키키      2
7307                 그러니까... 너무 좋아 돈 생겼다~=      0
7308                       우리 보컬 학원 다녀 볼래?      2
7309                    거기서도 딱 목소리가 들리고 좋아      2
7310            왜 안 믿어줘 다들... 정말이야 ... 너무해      1

[7311 rows x 2 columns]


In [ ]:
tokenizer = BertTokenizer.from_pretrained("klue/bert-base")

In [ ]:
max_seq_len = max(df['sentence'].apply(lambda x: len(str(x))))

sent1 = train_data['sentence'].iloc[3]
print(max_seq_len)
print('문 장1 :',sent1)

96
문 장1 : 오오 키키 그러면 뭐... 지각만 안 하면 괜찮겠는데요?


In [ ]:
encoding_result = tokenizer.encode(sent1)

In [ ]:
print(encoding_result)
tokenizer.decode(encoding_result)

[2, 22905, 1754, 2089, 5036, 1097, 18, 18, 18, 8955, 2154, 1378, 1889, 2460, 5110, 2918, 13964, 2182, 35, 3]


'[CLS] 오오 키키 그러면 뭐... 지각만 안 하면 괜찮겠는데요? [SEP]'

In [ ]:
encoded_result = tokenizer.encode(sent1,max_length=max_seq_len, pad_to_max_length=True)
print(encoded_result)
print('길 이 :', len(encoded_result))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[2, 22905, 1754, 2089, 5036, 1097, 18, 18, 18, 8955, 2154, 1378, 1889, 2460, 5110, 2918, 13964, 2182, 35, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
길 이 : 96


In [ ]:
# 세 그 멘 트 인 코 딩
print([0]*max_seq_len)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
# 어 텐 션 마 스 크 인 코 딩
valid_num = len(tokenizer.encode(sent1))
print(valid_num * [1] + (max_seq_len - valid_num) * [0])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
#입력된 전체 데이터에 대해서 이 과정을 진행하는 함수를 만듭니다.
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer):

    input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []

    for example, label in tqdm(zip(examples, labels), total=len(examples)):
    # input_id는 워 드 임 베 딩 을 위 한 문 장 의 정 수 인 코 딩
        input_id = tokenizer.encode(example, max_length=max_seq_len, pad_to_max_length=True)

        # attention_mask는 실 제 단 어 가 위 치 하 면 1, 패 딩 의 위 치 에 는 0인 시 퀀 스.
        padding_count = input_id.count(tokenizer.pad_token_id)
        attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count

        # token_type_id은 세 그 먼 트 인 코 딩
        token_type_id = [0] * max_seq_len

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention masklength {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length{} vs {}".format(len(token_type_id), max_seq_len)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        data_labels.append(label)

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)

    data_labels = np.asarray(data_labels, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), data_labels

In [ ]:
train_X, train_y = convert_examples_to_features(train_data['sentence'], train_data['intent'],
                                              max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 29248/29248 [00:05<00:00, 5250.63it/s]


In [ ]:
test_X, test_y = convert_examples_to_features(test_data['sentence'], test_data['intent'],
                                              max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 7311/7311 [00:01<00:00, 5469.33it/s]


In [ ]:
input_id = train_X[0][0]
attention_mask = train_X[1][0]
token_type_id = train_X[2][0]
label = train_y[0]

print('단 어 에 대 한 정 수 인 코 딩 :',input_id)
print('어 텐 션 마 스 크 :',attention_mask)
print('세 그 먼 트 인 코 딩 :',token_type_id)
print('각 인 코 딩 의 길 이 :', len(input_id))
print('정 수 인 코 딩 복 원 :',tokenizer.decode(input_id))
print('레 이 블 :',label)

단 어 에 대 한 정 수 인 코 딩 : [    2 20349    18    18    18     3     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0]
어 텐 션 마 스 크 : [1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
세 그 먼 트 인 코 딩 : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model = TFBertForSequenceClassification.from_pretrained("klue/bert-base", num_labels=lable_num, from_pt=True)
model.compile(optimizer=optimizer, loss=model.hf_compute_loss, metrics=['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.fit(train_X, train_y, epochs=8, batch_size=128, validation_split=0.2)

Epoch 1/8


InternalError: ignored

In [ ]:
results = model.evaluate(test_X, test_y, batch_size=32)
print("test loss, test acc: ", results)

In [ ]:
def sentiment_predict(new_sentence):
    input_id = tokenizer.encode(new_sentence, max_length=max_seq_len, pad_to_max_length=True)

    padding_count = input_id.count(tokenizer.pad_token_id)
    attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count
    token_type_id = [0] * max_seq_len

    input_ids = np.array([input_id])
    attention_masks = np.array([attention_mask])
    token_type_ids = np.array([token_type_id])

    encoded_input = [input_ids, attention_masks, token_type_ids]

    score = np.argmax(model.predict(encoded_input)[0])
    
    print('sentence :',new_sentence)
    print(label_dict[score])


In [ ]:
MODEL_NAME = 'fine-tuned-klue-bert-base'
MODEL_SAVE_PATH = os.path.join("/content/Drive/MyDrive/NP_classifier/update_labels_model", MODEL_NAME) # change this to your preferred location
print(MODEL_SAVE_PATH)
print('='*50)

if os.path.exists(MODEL_SAVE_PATH):
    print(f"{MODEL_SAVE_PATH} -- Folder already exists \n")

else:
    os.makedirs(MODEL_SAVE_PATH, exist_ok=True)
    print(f"{MODEL_SAVE_PATH} -- Folder create complete \n")

# save tokenizer, model
model.save_pretrained(MODEL_SAVE_PATH)
tokenizer.save_pretrained(MODEL_SAVE_PATH)

In [ ]:
new_sentence = input('sentence > ')
sentiment_predict(new_sentence)